# Creating chemical states

After defining and creating a [ChemicalSystem](https://reaktoro.org/api/classReaktoro_1_1ChemicalSystem.html) object, we are equipped to create chemical states. In applications, we often deal with either a *single chemical state* or *a collection of them* (as in reactive transport, for example) with fixed temperature, pressure, and species amounts specified for each state. [ChemicalState](https://reaktoro.org/api/classReaktoro_1_1ChemicalState.html) class is commonly use for:
* specifying *initial conditions* for chemical equilibrium and kinetics calculations
* *storing computed states* resulting from the above computations

First, we consider an example of the chemical system with a gaseous phase only and corresponding [ChemicalState](https://reaktoro.org/api/classReaktoro_1_1ChemicalState.html) instance:

In [1]:
from reaktoro import *

db = NasaDatabase("nasa-cea")

gases = GaseousPhase("CO2 O2 H2 H2O CH4 CO")

system = ChemicalSystem(db, gases)

# Providing initial chemical state
state = ChemicalState(system)
state.temperature(1000, "celsius") # remove this line to see the default value
state.pressure(10, "MPa")          # remove this line to see the default value
state.set("CO2", 0.1, "mol")
state.set("O2" , 0.2, "mol")
state.set("H2" , 0.3, "mol")
state.set("H2O", 0.4, "mol")
state.set("CH4", 0.5, "mol")
state.set("CO" , 0.0, "mol")       # remove this line to see the default value

We can print this chemical state to inspect its content:

In [2]:
print("INITIAL STATE")
print(state)

INITIAL STATE
+-----------------+------------+------+
| Property        |      Value | Unit |
+-----------------+------------+------+
| Temperature     |  1273.1500 |    K |
| Pressure        |   100.0000 |  bar |
| Charge:         | 0.0000e+00 |  mol |
| Element Amount: |            |      |
| :: H            | 3.4000e+00 |  mol |
| :: C            | 6.0000e-01 |  mol |
| :: O            | 1.0000e+00 |  mol |
| Species Amount: |            |      |
| :: CO2          | 1.0000e-01 |  mol |
| :: O2           | 2.0000e-01 |  mol |
| :: H2           | 3.0000e-01 |  mol |
| :: H2O          | 4.0000e-01 |  mol |
| :: CH4          | 5.0000e-01 |  mol |
| :: CO           | 0.0000e+00 |  mol |
+-----------------+------------+------+


The table above **does not** contain *thermodynamic properties of the system*; only temperature, pressure, species and element amounts, and the electric charge.


> **Note**: By default, a [ChemicalState](https://reaktoro.org/api/classReaktoro_1_1ChemicalState.html) object is initialized with the following conditions:
> * 273.15 K (or 25 °C)
> * 10<sup>5</sup> Pa (or 1 bar)
> * 10<sup>-16</sup> mol as the amounts of every species.
> Zero is a numerically problematic value for species amounts; that's why we set it to a very small positive value instead. This should not be an issue for you in most cases, but if for some reason 10<sup>-16</sup> is not small enough, you can use the method `ChemicalState.setSpeciesAmount(1e-40)` to set a common amount value for all species in the system.

**TASK**: 
1) consider a more complicated chemical system containing
* mineral phases of Calcite, Dolomite, Quartz, Halite
* aqueous phase generated automatically based on the elements contained in the mineral phases
and 
2) create a chemical state with the following initial composition of the species per 1 kg of water at 50 °C and 5 atm:

| Aqueous species | Value     |
|-----------------|-----------|
| Na<sup>+</sup>  | 0.10 mol  |
| Cl<sup>-</sup>  | 0.10 mol  |
| Mg<sup>2+</sup> | 0.50 mmol |
| Ca<sup>2+</sup> | 0.50 mmol |
| Calcite         | 1 mg      |
| Dolomite        | 1 ug      |
| Quartz          | 1 kmol    |


In [3]:
# -------------------------------------- #
# Code cell for the task
# -------------------------------------- #

db = PhreeqcDatabase("pitzer.dat")

solution = AqueousPhase()
minerals = MineralPhases("Halite Calcite Dolomite Quartz")

system = ChemicalSystem(db, solution, minerals)

state = ChemicalState(system)
state.temperature(50.0, "celsius")
state.pressure(5.0, "atm")
state.set("H2O"     , 1.00, "kg")
state.set("Na+"     , 0.10, "mol")
state.set("Cl-"     , 0.10, "mol")
state.set("Ca+2"    , 0.50, "mmol")
state.set("Mg+2"    , 0.50, "mmol")
state.set("Calcite" , 1.00, "mg")
state.set("Dolomite", 1.00, "ug")
state.set("Quartz"  , 1.00, "kmol")

print(state)

+---------------------------+------------+------+
| Property                  |      Value | Unit |
+---------------------------+------------+------+
| Temperature               |   323.1500 |    K |
| Pressure                  |     5.0663 |  bar |
| Charge:                   | 2.0000e-03 |  mol |
| Element Amount:           |            |      |
| :: H                      | 1.1101e+02 |  mol |
| :: C                      | 1.0002e-05 |  mol |
| :: O                      | 2.0555e+03 |  mol |
| :: Na                     | 1.0000e-01 |  mol |
| :: Mg                     | 5.0001e-04 |  mol |
| :: Si                     | 1.0000e+03 |  mol |
| :: Cl                     | 1.0000e-01 |  mol |
| :: Ca                     | 5.1000e-04 |  mol |
| Species Amount:           |            |      |
| :: H+                     | 1.0000e-16 |  mol |
| :: H2O                    | 5.5506e+01 |  mol |
| :: CO3-2                  | 1.0000e-16 |  mol |
| :: CO2                    | 1.0000e-16 |  mol |


## Setting surface area

Providing surface areas between some phases is an important functionality in modelling chemical kinetics in which heterogeneous reactions (i.e., reactions containing species from different phases, such as mineral and gas dissolution reactions) are common:

In [4]:
db = PhreeqcDatabase("pitzer.dat")

solution = AqueousPhase()
minerals = MineralPhases("Calcite Quartz") # HINT: adding Dolomote

system = ChemicalSystem(db, solution, minerals)

state = ChemicalState(system)
state.set("H2O"     , 1.00, "kg")

state.surfaceArea("AqueousPhase", "Calcite", 1.2, "m2")
state.surfaceArea("AqueousPhase", "Quartz" , 1.4, "m2")
# HINT: adding surface aread between Dolomote and AqueousPhase 

print(state)

+---------------------------+-------------+------+
| Property                  |       Value | Unit |
+---------------------------+-------------+------+
| Temperature               |    298.1500 |    K |
| Pressure                  |      1.0000 |  bar |
| Charge:                   | -4.0000e-16 |  mol |
| Element Amount:           |             |      |
| :: H                      |  1.1101e+02 |  mol |
| :: C                      |  4.0000e-16 |  mol |
| :: O                      |  5.5506e+01 |  mol |
| :: Si                     |  4.0000e-16 |  mol |
| :: Ca                     |  2.0000e-16 |  mol |
| Species Amount:           |             |      |
| :: H+                     |  1.0000e-16 |  mol |
| :: H2O                    |  5.5506e+01 |  mol |
| :: CO3-2                  |  1.0000e-16 |  mol |
| :: CO2                    |  1.0000e-16 |  mol |
| :: Ca+2                   |  1.0000e-16 |  mol |
| :: H4SiO4                 |  1.0000e-16 |  mol |
| :: H2SiO4-2               |  

Initially, [ChemicalState](https://reaktoro.org/api/classReaktoro_1_1ChemicalState.html) contains no surface areas. As you set surface areas for some phase pairs, interphase surface information is saved in the [ChemicalState](https://reaktoro.org/api/classReaktoro_1_1ChemicalState.html) object. To access the phase pairs for which surfaces areas have been set and retrieve their values can be done as follows:

In [5]:
print("EXISTING SURFACES IN THE CHEMICAL STATE OBJECT")
for [i, j] in state.surfaces():
    name1 = system.phase(i).name()      # the name of the first phase in the phase pair
    name2 = system.phase(j).name()      # the name of the other phase in the phase pair
    surfarea = state.surfaceArea(i, j)  # the surface area of this phase pair
    print(f"Surface area between {name1} and {name2}: {surfarea} m2")

EXISTING SURFACES IN THE CHEMICAL STATE OBJECT
Surface area between AqueousPhase and Calcite: 1.2 m2
Surface area between AqueousPhase and Quartz: 1.4 m2
